# LangSmith Tracking Setup

This Tutorial comes [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial) with minor Changes by Martin Fockedey

- Author: [JeongGi Park](https://github.com/jeongkpa)
- Peer Review: [MinJi Kang](https://www.linkedin.com/in/minji-kang-995b32230/), [Wooseok Jeong](https://github.com/jeong-wooseok)
- Proofread : [Q0211](https://github.com/Q0211)
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/01-Basic/04-LangSmith-Tracking-Setup.ipynb)[![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/01-Basic/04-LangSmith-Tracking-Setup.ipynb)
## Overview

This tutorial covers how to set up and use ```LangSmith```, a powerful platform for developing, monitoring, and testing LLM applications. 
```LangSmith``` provides comprehensive tracking capabilities that are essential for understanding and optimizing your LLM applications.

```LangSmith``` tracking helps you monitor:

- Token usage and associated costs
- Execution time and performance metrics
- Error rates and unexpected behaviors
- Agent interactions and chain operations

In this tutorial, we'll walk through the process of setting up ```LangSmith``` tracking and integrating it with your ```LangChain``` applications.

### Table of Contents

- [Overview](#overview)
- [Setting up a LangSmith trace](#setting-up-a-langsmith-trace)
- [Using LangSmith tracking](#using-langsmith-tracking)
- [Enable tracking in your Jupyter notebook or code](#enable-tracking-in-your-jupyter-notebook-or-code)

### References

- [OpenAI API Pricing](https://openai.com/api/pricing/)
- [Token Usage Guide](https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them)
- [LangChain Python API Reference](https://python.langchain.com/api_reference/community/callbacks/langchain_community.callbacks.manager.get_openai_callback.html)
---

## Setting up a LangSmith trace

```LangSmith``` is a platform for developing, monitoring, and testing LLM applications. 
If you're starting a project or learning ```LangChain```, ```LangSmith``` is a must-have to get set up and running.

### Project-Level Tracking
At the project level, you can check execution counts, error rates, token usage, and billing information.

![project-level-tracking](./assets/03-langsmith-tracking-setup-01.png)

When you click on a project, all executed Runs appear.

![project-level-tracking-detail](./assets/03-langsmith-tracking-setup-02.png)


### Detailed Step-by-Step Tracking for a Single Execution

![detailed-step-by-step-tracking](./assets/03-langsmith-tracking-setup-03.png)


After a single execution, it records not only the search results of retrieved documents but also detailed logs of GPT's input and output content. 
Therefore, it helps you determine whether to change the search algorithm or modify prompts after reviewing the searched content.


Moreover, at the top, it shows the time taken for a single Run (about 30 seconds) and tokens used (5,104), and when you hover over the tokens, it displays the billing amount.

## Using LangSmith tracking

Using traces is very simple.

### Get a LangSmith API Key


1. Go to https://smith.langchain.com/ and sign up.
2. After signing up, you will need to verify your email.
3. Click the left cog (Setting) - center "Personal" - "Create API Key" to get an API key.

![get-api-key](./assets/03-langsmith-tracking-setup-04.png)



In Description, enter a description that makes sense to you and click the Create API Key button.

![create-api-key](./assets/03-langsmith-tracking-setup-05.png
)


Copy the generated key and proceed to the next step.

(Caution!) Copy the generated key somewhere safe so that it doesn't leak.

![copy-api-key](./assets/03-langsmith-tracking-setup-06.png)



### Setting the LangSmith key in ```.env```


First, enter the key you received from LangSmith and your project information in the .env file.

- ```LANGCHAIN_TRACING_V2```: Set to "true" to start tracking.
- ```LANGCHAIN_ENDPOINT```: https://api.smith.langchain.com (Do not modify this value).
- ```LANGCHAIN_API_KEY```: Enter the key you received in the previous step.
- ```LANGCHAIN_PROJECT```: Specify a project name to group and trace all runs under that project group.

![setting-api-key](./assets/03-langsmith-tracking-setup-07.png)


## Enable tracking in your Jupyter notebook or code

Enabling tracking is very simple. All you need to do is set an environment variable.

Copy the contents of ```.env_sample``` and load it into your ```.env``` with the key you set

In [19]:
%%capture --no-stderr
%pip install python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Users\FKY\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [20]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

As long as your traces are enabled and your API key and project name are set correctly, tracking will work properly.

However, if you want to change the project name or change the tracking, you can do so with the code below.

In [21]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
#s.environ["LANGCHAIN_PROJECT"] = "<LangChain Project Name>"
#s.environ["LANGCHAIN_API_KEY"] = "<LangChain API KEY>"

## Practical Example: Tracking a Simple RAG System

Now that we have LangSmith configured, let's see it in action with a real RAG (Retrieval-Augmented Generation) system. This example is based on the simple RAG from notebook 07.

We'll build a basic RAG pipeline and use LangSmith to:
- **Trace** each step (document loading, splitting, retrieval, generation)
- **Monitor** performance and token usage
- **Debug** by inspecting retrieved documents and prompts
- **Evaluate** response quality over multiple queries

### Step 1: Install Required Packages

First, let's install the packages we need for our RAG system.

In [22]:
%%capture --no-stderr
%pip install -q langchain_mistralai langchain_community langchain_text_splitters langchain_core faiss-cpu pymupdf


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Users\FKY\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### Step 2: Build a Simple RAG Pipeline

We'll create a basic RAG system that loads the ECAM regulation PDF, splits it into chunks, creates embeddings, and answers questions.

In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings

# Step 1: Load Documents
loader = PyMuPDFLoader("RéglementECAM.pdf")
docs = loader.load()
print(f"✓ Loaded {len(docs)} pages from PDF")

# Step 2: Split Documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"✓ Split into {len(split_documents)} chunks")

# Step 3: Generate Embeddings and Create Vector Store
embeddings = MistralAIEmbeddings(model="mistral-embed")
STORE_DIR = 'faiss_store'
if os.path.isdir(STORE_DIR):
    print('Loading existing FAISS store...')
    vectorstore = FAISS.load_local(STORE_DIR, embeddings, allow_dangerous_deserialization=True)
else:
    print('Creating new FAISS store and saving...')
    vectorstore = FAISS.from_documents(split_docs, embeddings)
    vectorstore.save_local(STORE_DIR)

# Step 4: Create Retriever
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

# Step 5: Create Prompt
prompt = PromptTemplate.from_template(
    """You are an assistant for the school ECAM-ICHEC-ISFSC for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

# Step 6: Setup LLM
llm = ChatMistralAI(model="mistral-small-latest", temperature=0)

# Step 7: Create RAG Chain (This will be automatically traced by LangSmith!)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
)

print("\n✓ RAG pipeline created and ready!")
print("  All operations will be traced in LangSmith automatically.")

✓ Loaded 61 pages from PDF
✓ Split into 497 chunks
Loading existing FAISS store...

✓ RAG pipeline created and ready!
  All operations will be traced in LangSmith automatically.


### Step 3: Test the RAG with LangSmith Tracing

Now let's run some queries. **Each query will be automatically traced in LangSmith!**

Go to your LangSmith dashboard at https://smith.langchain.com to see the traces in real-time.

In [24]:
# Test with several questions
test_questions = [
    "Qu'est-ce que la financabilité d'un étudiant?",
    "Quelles sont les sanctions pour le plagiat?",
    "Comment créer mon programme d'étude personnalisé?",
]

print("Running queries with LangSmith tracing enabled...")
print("="*70)

for i, question in enumerate(test_questions, 1):
    print(f"\n📝 Question {i}: {question}")
    print("-"*70)
    
    response = rag_chain.invoke(question)
    print(f"🤖 Answer: {response[:200]}...")
    print()

print("="*70)
print("✅ All queries traced successfully!")
print("🔍 Check your LangSmith dashboard to see:")
print("   - Retrieved documents for each query")
print("   - Complete prompt sent to the LLM")
print("   - Token usage and costs")
print("   - Execution time for each component")
print(f"\n🌐 Dashboard: https://smith.langchain.com")

Running queries with LangSmith tracing enabled...

📝 Question 1: Qu'est-ce que la financabilité d'un étudiant?
----------------------------------------------------------------------
🤖 Answer: La **finançabilité** d'un étudiant fait référence à l'éligibilité financière pour bénéficier d'une aide ou d'un soutien financier de la part de la Fédération Wallonie-Bruxelles (ou d'autres instances)...


📝 Question 2: Quelles sont les sanctions pour le plagiat?
----------------------------------------------------------------------
🤖 Answer: Les sanctions pour le plagiat à l'ECAM-ICHEC-ISFSC incluent :

1. **Sanction disciplinaire** : La Direction du Département peut prononcer une sanction disciplinaire conformément à l'article 100 du règ...


📝 Question 3: Comment créer mon programme d'étude personnalisé?
----------------------------------------------------------------------
🤖 Answer: Pour créer votre programme d'étude personnalisé à l'ECAM-ICHEC-ISFSC, voici les étapes à suivre selon le context

### Step 4: Understanding the LangSmith Trace

When you look at a trace in LangSmith, you'll see:

1. **Overall Execution**:
   - Total time taken
   - Total tokens used
   - Cost estimation
   - Success/Error status

2. **Step-by-Step Breakdown**:
   - **Retrieval**: Which documents were retrieved from the vector store
   - **Prompt Construction**: The exact prompt sent to the LLM (with context)
   - **LLM Call**: Input tokens, output tokens, model used
   - **Output Parsing**: Final formatted response

3. **Metadata**:
   - Timestamp
   - Model version
   - Tags and custom metadata (if added)

   By exposing each component’s inputs, outputs, timings, and token usage, the trace lets you quickly debug incorrect answers, optimize slow or costly steps, monitor performance and spend over time, and iteratively improve quality by comparing different prompts, models, or retrieval settings side‑by‑side.

### Step 5:Controlling Tracing

Sometimes you may want to temporarily disable or re-enable tracing.

In [ ]:
# Disable tracing
print("1️⃣ Disabling tracing...")
os.environ["LANGCHAIN_TRACING_V2"] = "false"

response = rag_chain.invoke("Qu'est-ce qu'une UE?")
print(f"   Response (NOT traced): {response[:80]}...\n")

# Re-enable tracing
print("2️⃣ Re-enabling tracing...")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

response = rag_chain.invoke("Qu'est-ce qu'une AA?")
print(f"   Response (TRACED): {response[:80]}...\n")


1️⃣ Disabling tracing...
   Response (NOT traced): Une **unité d’enseignement (UE)** est un élément constitutif d'un programme d'ét...

2️⃣ Re-enabling tracing...
   Response (TRACED): D'après le contexte fourni, "AA" n'est pas explicitement défini. Cependant, dans...

✅ Tracing control demonstrated!

💡 Use cases for disabling tracing:
   - High-volume production queries (to reduce overhead)
   - Privacy-sensitive queries
   - Performance testing without observability overhead
   - Development/testing scenarios
